In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [4]:
df = pd.read_sql("""
SELECT patient_id, sex, birth_date, created_at
FROM patient
ORDER BY patient_id;
""", engine)

df


,patient_id,sex,birth_date,created_at
0,1,F,1990-05-12,2026-01-15 19:15:06.417200
1,2,M,1984-11-03,2026-01-15 19:15:06.417200
2,3,O,2001-07-21,2026-01-15 19:15:06.417200


In [5]:
pd.read_sql("SELECT sex, COUNT(*) n FROM patient GROUP BY sex ORDER BY n DESC;", engine)


,sex,n
0,O,1
1,M,1
2,F,1


In [4]:
df = pd.read_sql(""" SELECT patient_id, sex, birth_date, city, has_chronic_condition, created_at FROM patient ORDER BY patient_id; """, engine)

df


,patient_id,sex,birth_date,city,has_chronic_condition,created_at
0,1,F,1990-05-12,Guatemala,True,2026-01-16 00:38:34.253802
1,2,M,1984-11-03,Antigua,False,2026-01-16 00:38:34.253802
2,3,O,2001-07-21,Quetzaltenango,True,2026-01-16 00:38:34.253802
3,4,F,1975-02-18,Guatemala,False,2026-01-16 00:38:34.253802
4,5,M,1998-09-30,Escuintla,False,2026-01-16 00:38:34.253802


In [5]:
sex_count = pd.read_sql(""" SELECT sex, COUNT(*) AS COUNT FROM patient GROUP BY sex """, engine)
sex_count

,sex,count
0,O,1
1,M,2
2,F,2


In [12]:
cond_count = pd.read_sql(""" SELECT COUNT(*) AS COUNT FROM patient WHERE has_chronic_condition=true; """, engine)
cond_count

,count
0,2
